# Non-planar ironing script


### Input Data

This script creates a non planar ironing layer on top of a 3d print as a proof of concept. Given there are no rises in the y plane, then ironing can be achieved by splitting a model into rectangles, zigzagging back and forth between 2 y coordinates while incrementing the z and x coordinates.

The input data need is lines of g-code in the following format

```
G1 X0 Y1 Z0 E0  ; FOR Y VALUE
G1 X2 Y2 Z2 E0
G1 X3 Y2 Z3 E0
```

* The value of E does not matter
* All lines should be G1 Commands
* The first line is only used for getting the secondary y value
* all other lines generate a 2d curve and so the printer and follow the curve while oscillating between the 2 y values.


In [86]:
# reads file and outputs data so you can ensure the data is correct

line_gap = 0.05 # adjust for different ironing smoothness

with open('input.gcode') as f:
    lines = f.readlines()
    
    print(lines)

['G1 X0 Y107.280 Z0 E0\n', 'G1 X90.220 Y92.720 Z11.940 E0\n', 'G1 X109.725 Y92.775 Z2.160 E0']


### Finds 2 Y values to bounce between

In [87]:
line = lines[0].split(' ')

y0 = float(line[2].replace('Y' ,''))

line = lines[1].split(' ')
y1 = float(line[2].replace('Y' ,''))
lines.pop(0)

print(y0, y1)


107.28 92.72


### Generates an array of the needed coordinates

Bounce between y while moving along the xz path, using the helper class point, and adding them to an array

In [88]:
from point import Point

grid = []
y = y0

for line in lines:
    y = y0 if(y == y1) else y1


    line = line.split(' ')
    x= (float(line[1].replace('X' ,'')))
    z = (float(line[3].replace('Z' ,'')))

    point = Point(x, y, z, 0)

    grid.append(point)
    
print(lines)
    

['G1 X90.220 Y92.720 Z11.940 E0\n', 'G1 X109.725 Y92.775 Z2.160 E0']


### Generate g code

generates the final g-code for the ironing

In [89]:
import copy


z_offset = 0 # offset/inset from surface

g_code = []


for i in range(len(grid) - 1):
    corner_a = grid[i]
    corner_b = grid[i + 1]

    # Go towards origin preventing scraping issues
    [corner_b, corner_a] = corner_a.smaller(corner_b)

    # find the number of lines needed to move between points using teh specified line_gap
    number_of_zags = abs((corner_a.x-corner_b.x))/line_gap
    z_gap = -((corner_a.z-corner_b.z)/number_of_zags)

    # deep copy start of rectangle
    current = copy.deepcopy(corner_a)
    current.z += z_offset
    

    # zigzag until made it to final point
    while(current.x > corner_b.x):
        g_code.append(copy.deepcopy(current))
        current.x -= line_gap
        current.z += z_gap
        g_code.append(copy.deepcopy(current))
        current.y = corner_a.y if (corner_a.y != current.y ) else corner_b.y
       
# Lift extruder up
final_point = copy.deepcopy(g_code[-1]) 
final_point.z = 20;
g_code.append(final_point)
        

### Write g code to file

In [90]:
with open('output.gcode', "w") as f:
    f.write("; line gap ="+ str(line_gap)+ "\n")
    
    
    for(i, point) in enumerate(g_code):
        lines = f.write(str(point) + "\n")
    
    

### Add gcode back into sliced version and view on a g code visualizer